In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from astropy.io import fits

In [2]:
#Needed Data
path = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2458098'
nights = sorted(glob('/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/2*')) 
fitsfiles = sorted(glob(nights[0] + '/*/*image.image.fits'))
muellermodel = '/lustre/aoc/projects/hera/gtucker/notebooks/muellerbeam.fits'
#muellermodels = sorted(glob('simfits/*'))

In [3]:
# Let's assumed we have a stack of maps, each of 4 pseudo Stokes parameters, 
# and a corresponding stack of Mueller matrices
#nmaps =  number of different times for which we made a map... ie time integrations(LST)
nx = 512
ny = 512
nmaps = len(fitsfiles)
pStokes = np.ones([nx,ny,4,nmaps])
Mueller = np.ones([nx,ny,4,4,nmaps])
Stokes = np.ones([nx,ny,4,nmaps])

In [4]:
# We will create the Mueller area by using the mueller sim, and using np.broadcast to
#'repeat' the array nmap times
hdu = fits.open(muellermodel)
data = hdu[0].data
#This gets rid of the frequency axis since we don't need it for now
new_data = np.squeeze(data,axis=2).T
#Repeats the (512,512,4,4) nmaps time... ie adds 5th dimension of Mueller array
Mueller= np.broadcast_to(new_data[...,None],(nx,ny,4,4,nmaps))

In [5]:
#Create pStokes
dirty_images = np.ones((nmaps,4,nx,ny)) #array holding all images for one night
i = 0 #needed to loop over every image in night
for file in fitsfiles:
    hdu = fits.open(file)
    #Copies over I,Q,U,V stokes to dirty_images 
    dirty_images[i][0] = hdu[0].data[0]
    dirty_images[i][1] = hdu[0].data[1]
    dirty_images[i][2] = hdu[0].data[2]
    dirty_images[i][3] = hdu[0].data[3]
    i += 1
pStokes = dirty_images.T

In [6]:
print(Mueller.shape)
print(Stokes.shape) #at this point, still full of 1's
print(pStokes.shape)

(512, 512, 4, 4, 56)
(512, 512, 4, 56)
(512, 512, 4, 56)


In [7]:
# We want to preserve the ability to do the Mueller inversion on just one map, too ...

In [8]:
# There's probably a faster way, but the brute force approach is to construct the linear combination at every pixel
inx = 0
iny = 0
ndata = 4*nmaps

In [9]:
# Construct A and the data
A = np.zeros([ndata,4])
d = np.zeros([ndata])
for imap in np.arange(nmaps):
    indx = np.arange(imap*4, (imap+1)*4)
    A[indx, :] = Mueller[inx, iny, :, :, imap]
    d[indx] = pStokes[inx, iny, :, imap]

In [10]:
print(A.shape) #this is the 4x4 mueller beam at each pixel
print(d.shape) #this is the data at each pixel

(224, 4)
(224,)


In [11]:
# We will probably always assume W is diagonal; but there might be noise correlation between the pseudo-Stokes ... ? 
W = np.ones([ndata,ndata])

In [12]:
# Check that everything is the right shape
ATWd = np.matmul(A.T, np.matmul(W,d))
Cov = np.matmul(A.T, np.matmul(W, A))
# Need to think about computing the condition number and the inversion method here
invCov = Cov # np.linalg.inv(Cov)

In [13]:
bestfit = np.matmul(invCov, ATWd)
print(bestfit)

[ 3.00932042e-05  2.23423563e-05 -2.11862012e-05  4.00391127e-05]


In [14]:
Stokes[inx, iny, ]

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1